In [1]:
import os
import sys
import random

import pandas as pd
import numpy as np

import matplotlib
matplotlib.use('AGG')
import matplotlib.pyplot as plt

import json
import argparse

from utils import resize_image, lovasz_loss, iou, iou_2, iou_metric_batch, cov_to_class, log_metric, rle_encode, predict_result
from models import build_unet_resnet_model, build_pretrained_model, unfreeze_weights, preprocess_input

from sklearn.model_selection import train_test_split, StratifiedKFold

from tqdm import tqdm

from keras.models import Model, load_model, save_model
from keras.layers import Input,Dropout,BatchNormalization,Activation,Add
from keras.layers.core import Lambda
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, TensorBoard
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
from keras import optimizers

import tensorflow as tf

from keras.preprocessing.image import array_to_img, img_to_array, load_img

Using TensorFlow backend.


In [2]:
from segmentation_models import Unet
from segmentation_models.backbones import get_preprocessing

backbones = ['vgg19','resnet34','resnet50','resnet152','resnext50','resnext101','densenet121','densenet169','densenet201','inceptionv3','inceptionresnetv2']
types = ['Unet','FPN']

rows_list = []
for backbone in backbones:
    for type in types:
        
        dict1 = {'type':type, 'backbone':backbone}
        # prepare model
        model = build_pretrained_model(type, backbone, None, False, 'sigmoid')
        dict1['input_shape'] = model.layers[0].input_shape
        dict1['output_shape'] = model.layers[-1].output_shape
        dict1['activation'] = model.layers[-1].activation
        dict1['params'] = model.count_params()
        rows_list.append(dict1)
        
df = pd.DataFrame(rows_list)
df

,activation,backbone,input_shape,output_shape,params,type
0,<function sigmoid at 0x000001CCDD21A9D8>,vgg19,"(None, None, None, 3)","(None, None, None, 1)",24348497,Unet
1,<function sigmoid at 0x000001CCDD21A9D8>,vgg19,"(None, None, None, 3)","(None, None, None, 21)",24718933,FPN
2,<function sigmoid at 0x000001CCDD21A9D8>,resnet34,"(None, None, None, 3)","(None, None, None, 1)",24456154,Unet
3,<function sigmoid at 0x000001CCDD21A9D8>,resnet34,"(None, None, None, 3)","(None, None, None, 21)",25784030,FPN
4,<function sigmoid at 0x000001CCDD21A9D8>,resnet50,"(None, None, None, 3)","(None, None, None, 1)",32561114,Unet
5,<function sigmoid at 0x000001CCDD21A9D8>,resnet50,"(None, None, None, 3)","(None, None, None, 21)",28764894,FPN
6,<function sigmoid at 0x000001CCDD21A9D8>,resnet152,"(None, None, None, 3)","(None, None, None, 1)",67295194,Unet
7,<function sigmoid at 0x000001CCDD21A9D8>,resnet152,"(None, None, None, 3)","(None, None, None, 21)",63498974,FPN
8,<function sigmoid at 0x000001CCDD21A9D8>,resnext50,"(None, None, None, 3)","(None, None, None, 1)",32063194,Unet
9,<function sigmoid at 0x000001CCDD21A9D8>,resnext50,"(None, None, None, 3)","(None, None, None, 21)",28266974,FPN
